In [ ]:
import numpy as np
import pandas as pd
import trackpy
import os
import h5py
import matplotlib as mpl
import matplotlib.pyplot as plt
from datetime import datetime
import mspt.particle_fitting as fit
import mspt.image_processing as img
import mspt.particle_detection as detect
import mspt.trajectory_analysis1 as traj
import mspt.plotting as plot

os.environ['NUMEXPR_MAX_THREADS'] = '32'

# General parameters
batch_mode = True
frame_range = []
navg = 1
mode = 'continuous_median'
window_length = 1001
parallel = True
GPU = False
thresh = 0.00055
DoG_estimates = {'T': 0.1423, 's': 2.1436, 'sigma': 1.2921}
dmax = 4.0
max_frames_to_vanish = 0
minimum_trajectory_length = 5

# Movie acquisition parameters
frame_rate = 199.8
pixel_size = 84.4
n_timelags_MSD = None
n_timelags_JDD = None
slope = 1
offset = 0

# Specify directory
directory = img.directoryDialog(os.getcwd())
filepaths_data = img.find_filepaths(directory, extension='mp', exclude=None)

assert len(filepaths_data) > 0, 'Cannot find any movie files to process in current directory'

problem_files = []

# Process each file in the directory
for filename in filepaths_data:
    try:
        print(f"Processing file: {filename}")
        # Apply continuous median background removal
        frames, file = img.mp_reader(
            batch_mode=batch_mode,
            file_to_load=filename,
            frame_range=frame_range,
            mode=mode,
            navg=navg,
            window_length=window_length,
            parallel=parallel,
            GPU=GPU
        )
        print("Background removal completed.")

        # Detect and fit candidate spots
        fitted_particles = fit.particle_fitter(
            frames,
            halfsize=window_length // 2,
            thresh=thresh,
            method='trust-ncg',
            DoG_estimates=DoG_estimates
        )
        print("Candidate spots detection and fitting completed.")

        # Check if there are any valid particles
        if fitted_particles.empty:
            print("No particles detected, generating empty plot.")
            fig, ax = plt.subplots(figsize=(5, 5))
            ax.text(0.5, 0.5, 'No particles detected', horizontalalignment='center', verticalalignment='center', fontsize=12)
            ax.set_title('No particles detected')
            plot_filename = os.path.join(directory, os.path.splitext(os.path.basename(filename))[0] + '_MSPT_plot.pdf')
            fig.savefig(plot_filename, transparent=True, bbox_inches='tight')
            plt.close(fig)
            continue

        # Link trajectories
        linked_trajectories = trackpy.link_df(fitted_particles, search_range=dmax, memory=max_frames_to_vanish)
        trajectories_lenfilt = trackpy.filter_stubs(linked_trajectories, minimum_trajectory_length)
        print("Trajectories linked and filtered.")

        # Save trajectories to a temporary CSV file
        trajectories_file = 'trajectories_temp.csv'
        trajectories_lenfilt.to_csv(trajectories_file, index=False)
        print(f"Trajectories saved to {trajectories_file}.")

        # Fit trajectories and store results in memory
        dummy_output_file = 'dummy_output.h5'
        traj.fit_trajectories(
            [trajectories_file],
            output_file=dummy_output_file,
            frame_rate=frame_rate,
            pixel_size=pixel_size
        )
        print("Trajectories fitting completed.")

        # Load data and convert contrast to mass
        with pd.HDFStore(dummy_output_file, 'r') as dfs:
            results = pd.concat([dfs[key] for key in dfs.keys()])
        os.remove(dummy_output_file)
        os.remove(trajectories_file)
        print("Data loaded from dummy output file and cleaned up.")

        results['median_mass'] = traj.apply_calibration(results, slope=slope, offset=offset)
        print("Converted contrast to mass.")

        # Plot data
        fig, axs, df_kde, dat = plot.generate_2D_KDE(
            results,
            x='median_mass',
            y='Deff_MSD_JDD',
            x_range=(0, 0.01),
            y_range=(-1, 1),  # in log space
            figsize=(5, 5),
            traj_length=5,
            density=None,
            n_levels=12,
            cmap=mpl.cm.gray_r,
            alpha=1.0,
            show=False
        )
        plot_filename = os.path.join(directory, os.path.splitext(os.path.basename(filename))[0] + '_MSPT_plot.pdf')
        fig.savefig(plot_filename, transparent=True, bbox_inches='tight')
        plt.close(fig)
        print(f"Plot saved to {plot_filename}.")

    except Exception as e:
        print(f"Error processing file {filename}: {e}")
        problem_files.append(filename)

print("Processing completed.")
if problem_files:
    print(f"Encountered issues with the following files: {problem_files}")


Frame 30745: 5 trajectories present.
